In [ ]:
import os
import tensorflow as tf
import numpy as np
from PIL import Image, ImageOps  # pip install pillow
import h5py
import zipfile

print("cwd:", os.getcwd())
print("files (top-level):", os.listdir('.'))

# Allow passing explicit model path via env var
model_path = os.getenv('MODEL_PATH')  # optional: set MODEL_PATH to full model file

def find_models(root='.'):
    exts = ('.keras', '.h5')
    for dirpath, _, files in os.walk(root):
        for f in files:
            if f.lower().endswith(exts):
                yield os.path.join(dirpath, f)

def is_ignored_path(p):
    ignore_tokens = (os.sep + '.venv' + os.sep, 'site-packages', os.sep + 'tests' + os.sep)
    return any(tok in p for tok in ignore_tokens)

def is_valid_keras_file(p):
    p_low = p.lower()
    if p_low.endswith('.h5'):
        try:
            with h5py.File(p, 'r') as f:
                return 'model_config' in f.attrs or 'layer_names' in f or 'keras_version' in f.attrs
        except Exception:
            return False
    if p_low.endswith('.keras'):
        try:
            if not zipfile.is_zipfile(p):
                return False
            with zipfile.ZipFile(p, 'r') as z:
                names = z.namelist()
                return 'keras_metadata.pb' in names or 'saved_model.pb' in names
        except Exception:
            return False
    return False

# Load model: try MODEL_PATH, then search common candidates and attempt to load valid-looking files
model = None
if model_path and os.path.exists(model_path):
    print('Cargando modelo desde MODEL_PATH:', model_path)
    model = tf.keras.models.load_model(model_path, compile=False)
else:
    candidates = list(find_models('.'))
    # sort: prefer current dir and avoid venv/site-packages
    candidates = sorted(candidates, key=lambda p: (is_ignored_path(p), os.path.dirname(p) != '.', p))
    if not candidates:
        raise FileNotFoundError(
            "Model file not found. Place 'mi_modelo.keras' in the notebook folder, or set MODEL_PATH to the correct file."
        )
    print('Model candidates found (ordered):')
    for c in candidates:
        size = os.path.getsize(c) if os.path.exists(c) else 0
        print(f'  - {c}  size={size} bytes  ignored={is_ignored_path(c)}  quick-valid={is_valid_keras_file(c)}')
    for candidate in candidates:
        if not is_valid_keras_file(candidate):
            print('Skipping (does not look like Keras model):', candidate)
            continue
        try:
            print('Attempting to load:', candidate)
            model = tf.keras.models.load_model(candidate, compile=False)
            print('Model loaded from:', candidate)
            break
        except Exception as e:
            print('Failed to load with tf.keras:', candidate, '->', e)
    if model is None:
        raise FileNotFoundError(
            "Found .h5/.keras files but none contained a valid Keras model. Place 'mi_modelo.keras' in the notebook folder or set MODEL_PATH."
        )

def predecir_ropa(ruta_imagen):
    # Preprocess image to match training: grayscale, invert, resize, normalize, batch
    img = Image.open(ruta_imagen).convert('L')
    img = ImageOps.invert(img)
    img = img.resize((28, 28))
    img_array = np.array(img) / 255.0
    img_array = img_array.reshape(1, 28, 28, 1)

    # Predict (use model loaded above)
    if model is None:
        raise RuntimeError('Model is not loaded')
    preds = model.predict(img_array)  # shape (1,10)
    probs = np.asarray(preds)[0]

    clases = ['Camiseta', 'Pantalón', 'Jersey', 'Vestido', 'Abrigo',
              'Sandalia', 'Camisa', 'Zapatilla', 'Bolso', 'Bota']
    idx = int(np.argmax(probs))
    conf = float(np.max(probs))

    print('------------------------------------------------')
    print('Imagen:', ruta_imagen)
    print('Predicción:', clases[idx].upper())
    print('Confianza: {0:.2f}%'.format(conf * 100))
    print('------------------------------------------------')
    return clases[idx], conf

In [ ]:
# Example usage (adjust path to your image).
try:
    predecir_ropa('zapato_mike.png')
except Exception as e:
    print('Error al predecir:', e)